# arXiv data subset eda

In [1]:
import os
import sys
parent_dr = os.path.split(os.getcwd())[0]
if parent_dr not in sys.path:
    sys.path.append(parent_dr)

In [13]:
import pandas as pd
from core.util.basic_io import *
from core.util.plot_util import *
from core.data.arxiv_data_io import *

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
file_name = "arxiv_subset_15540.json"
full_path = os.path.join(parent_dr, "core", "resources", file_name)
data = read_json_to_dict(full_path)

In [8]:
data_by_cat = {}
for article in data:
    all_cats = article["categories"]
    article_list = data_by_cat.get(all_cats, [])
    article_list.append(article)
    data_by_cat[all_cats] = article_list

In [10]:
cat_descriptions = []
for cat_str in data_by_cat:
    cat_list = cat_str.split(" ")
    for cat in cat_list:
        brief_desc = CATEGORY_DICT[cat] if cat in CATEGORY_DICT else "NONE FOUND"
        long_desc = CATEGORY_DESCR[cat] if cat in CATEGORY_DESCR else "NONE FOUND"
        cat_descriptions.append([cat, brief_desc, long_desc])

In [19]:
category_df = pd.DataFrame(cat_descriptions, columns=['Code', 'Category', 'Descr'])
category_df.sort_values(by=['Code'], inplace=True)
category_df

,Code,Category,Descr
1,cs.AI,Artificial Intelligence,"Covers all areas of AI except Vision, Robotics, Machine Learning,Multiagent Systems, and Computation and Language (Natural Language Processing), which have separate subject areas. In particular, includes Expert Systems, Theorem Proving (although this may overlap with Logic in Computer Science), Knowledge Representation, Planning, and Uncertainty in AI."
3,cs.CY,Computers and Society,"Covers impact of computers on society, computer ethics, information technology and public policy, legal aspects of computing, computers and education."
2,cs.DM,Discrete Mathematics,"Covers combinatorics, graph theory, applications of probability."
5,econ.TH,Theoretical Economics,"Includes theoretical contributions to Contract Theory, Decision Theory, Game Theory, General Equilibrium, Growth, Learning and Evolution, Macroeconomics, Market and Mechanism Design, and Social Choice."
6,eess.AS,Audio and Speech Processing,"Theory and methods for processing signals representing audio, speech, and language, and their applications. This includes analysis, synthesis, enhancement, transformation, classification and interpretation of such signals as well as the design, development, and evaluation of associated signal processing systems. Machine learning and pattern analysis applied to any of the above areas is also welcome."
0,q-bio.NC,Neurons and Cognition,"Synapse, cortex, neuronal dynamics, neural network, sensorimotor control, behavior, attention"
4,q-bio.TO,Tissues and Organs,"Blood flow in vessels, biomechanics of bones, electrical waves, endocrine system, tumor growth"
